<a href="https://colab.research.google.com/github/RulLu16/ML_practice/blob/master/%EC%8B%A4%EC%8A%B52.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 2 - Logistic Regression by 손병호(soMLier)
이번 실습에서는 Logistic Regression을 이용하여 간단한 데이터를 학습(train)하고 평가(test)하는 코드를 작성한다.

logistic regression의 기초적인 내용과 관련해서는 다음의 링크를 참고하면 도움이 될 것이다.

https://towardsdatascience.com/logistic-regression-explained-and-implemented-in-python-880955306060

In [0]:
import numpy as np

먼저 우리의 logistic regression 모델의 hyperparameter 들을 정의하자.
* x_dim : feature 벡터의 차원수
* lr : learning rate이다.

In [0]:
# model settings
x_dim = 2
lr = 0.01

모델이 사용할 data set이 필요하다. 실습에서는 실제 데이터를 사용하지 않고 generate_toy_data() 함수를 이용하여 인위적으로 만들 것이다.

data set은 여러개의 example들로 구성되고, 각 example은 다시 다음으로 구성된다.
* x : feature vector (특징벡터 = 독립변수)
* y : label (라벨 = 종속변수)

여기서 x는 2차원 벡터이고, y는 스칼라이다.

logistic regression은 기본적으로 binary classification 모델이므로, y는 0 혹은 1만을 가질 수 있다.

주목해야할 점은, y의 값에 따라 x의 분포가 다르다는 것이다.

In [0]:
def generate_toy_data(size):
  '''
    generates the toy data consisting of positive(50%) and negative(50%) examples
      positive mean : [1,1]
      negative mean : [-1,-1]
    Arguments:
      size -- number of positive examples(or negative examples)
    Returns:
      the tuple [feature vectors,labels]
  '''
  x0 = np.random.normal(size=2*size).reshape(-1, 2) - 1
  x1 = np.random.normal(size=2*size).reshape(-1, 2) + 1.
  return np.concatenate([x0, x1]), np.concatenate([np.zeros(size), np.ones(size)]).astype(np.int)

generate_toy_data() 함수를 이용하여 만들어낸 데이터의 모습을 관찰해보자.

generate_toy_data()로부터 생성된 X는 N개의 feature vector를 담고 있는 Matrix이고, y는 N개의 라벨을 담고 있는 vector 형태임을 확인할 수 있다.

실제로, 이렇게 X와 y를 분리시켜 각각이 여러개의 example들을 담고 있게끔 데이터를 보관하는 방법이 널리 쓰인다. 이러한 데이터(example들의 모음)를 batch라고 부르며, 머신러닝의 학습은 대부분 batch 단위로 수행된다. (위 설명에서 batch size는 N이 된다)

In [0]:
# prepare the data
train_X, train_y = generate_toy_data(30)
test_X, test_y = generate_toy_data(10)

print(test_X)
print(test_y)

[[-9.89894942e-01 -8.50125743e-01]
 [-1.78620641e-03 -2.56553418e+00]
 [-1.43921257e+00 -7.74665067e-01]
 [-1.15191031e+00 -1.54143926e+00]
 [-7.24646808e-01 -1.85212128e+00]
 [-2.08625117e+00 -5.88638151e-01]
 [-5.51936904e-01 -1.32899340e+00]
 [-1.38620673e+00 -1.40358714e+00]
 [-7.35798017e-01 -1.58112008e+00]
 [-4.00731707e+00  1.26620436e-01]
 [ 1.12265146e+00 -1.77313033e-01]
 [ 2.03923441e+00  8.27440209e-01]
 [ 9.36326366e-01  1.48962437e+00]
 [ 1.42342262e+00  7.53052581e-01]
 [ 1.43579159e+00  1.06926948e+00]
 [ 1.44591306e+00  2.43861324e+00]
 [ 5.16466145e-01  1.58095396e+00]
 [ 1.24367023e+00  1.66067536e+00]
 [-1.46880704e-01 -8.45722334e-01]
 [ 2.59703016e+00  2.04130054e+00]]
[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]


이제부터 logistic regression 모델을 학습 및 평가하는데 이용될 함수들을 정의할 것이다. 이 과정에서 input과 output 변수의 size에 주목하면 코드를 이해하는 것이 훨씬 수월해질 것이다.

## sigmoid

In [0]:
def sigmoid(z):
  """
    computes the sigmoid elementwise
    Arguments:
      z -- vector of size [batch]
    Returns:
      sigmoid(z) -- vector of size [batch]
  """
  return 1/(1+np.e**(-z))

## forward
logistic regression이 데이터 X를 가지고 결과 y_hat을 계산하는 과정을 그대로 따른다.

In [0]:
def forward(w,b,X):
  """
    computes the logistic regression value for the given input
    Arguments:
      w -- weights of size [x_dim]
      b -- bias, a scalar
      X -- input data of size [batch, x_dim]
    Returns:
      sigmoid(z) -- probabilities of being in class 1 of size [batch]
  """
  z = b + np.matmul(w,X.T)
  return sigmoid(z)

## loss
logistic regression의 오차함수이다.

In [0]:
def loss(y,y_hat):
  """
    computes the MSE loss
    Arguments:
      y -- the correct labels of size [batch]
      y_hat -- the predictions of size [batch]
    Returns:
      the MSE loss w.r.t. y_hat
  """
  return np.sum((y-y_hat)**2)/len(y)

## optimize
optimize() 함수는 우리의 logistic regression 모델을 이용하여 gradient descent를 "한 스텝" 진행시켜 준다. 이는 세부적으로 다음의 과정으로 진행된다.

1) W,b의 그래디언트 계산

2) W,b의 업데이트

In [0]:
def optimize(w,b,X,y,y_hat,lr=0.001):
  """
    optimize one step
    Arguments:
      w -- weights of size [x_dim]
      b -- bias, a scalar
      X -- input data of size [batch, x_dim]
      y -- labels of size [batch]
      y_hat -- predictions of size [batch]
      lr -- learning rate
    Returns:
      w -- optimized weights
      b -- optimized bias
  """
  bsize = len(y) # get batch size

  # dw -- gradient of w
  dw = np.zeros_like(w)
  for i in range(len(dw)):
    # TODO - calculate the derivative of L w.r.t w[i] (replace the 0)
    dw[i] = 0

  # db = get derivative at b
  # TODO - calculate the derivative of L w.r.t b (replace the 0)
  db = 0 

  # take a step
  w = w - lr * dw
  b = b - lr * db
  return w,b

## train
앞에서 만든 optimize() 함수를 반복적으로 여러번 수행한다. 즉 gradient descent 여러 스텝을 수행한다. 그 결과로 우리는 최적화된 모델 W,b를 얻을 것이라고 기대할 수 있다.

In [0]:
def train(w,b,X,y,epoch,lr=0.001):
  for _ in range(epoch):
    y_hat = forward(w,b,X)
    print(loss(y,y_hat))
    w,b = optimize(w,b,X,y,y_hat,lr=lr)
  return w,b

## eval
모델을 최종 평가하는 함수이다. 이때 위에서 정의한 loss를 이용하여 평가를 하지 않고, 진정한 의미의 성능 측도인 accuracy (correct/total)을 계산한다.

In [0]:
def eval(w,b,X,y):
  """
  returns the accuracy of the learned model
  """
  y_hat = forward(w,b,X)
  res = [0 if i<0.5 else 1 for i in y_hat]
  res = (res == y)
  res = np.sum(res)/len(y)
  print("accuracy : "+str(res))
  return res

## main

In [0]:
# initialize model by 0
weights = np.zeros(x_dim)
bias = 0.
print(weights, bias)
eval(weights, bias, test_X, test_y)

# train the model
weights, bias = train(weights,bias,train_X,train_y,100,lr=0.2)
print(weights, bias)

[0. 0.] 0.0
accuracy : 0.5
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25
[0. 0.] 0.0


In [0]:
# test the model
eval(weights, bias, test_X, test_y)

accuracy : 0.5


0.5